# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [14]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [15]:
path = "datafiles/City_Data.csv"
vacay_df = pd.read_csv(path)
vacay_df.head()

,city,country,id,lat,long,temp,max_temp,humidity,cloud_coverage,wind_speed
0,Jamestown,US,5122534,42.10,-79.24,73.08,73.99,94,75,10.29
1,Parang,PH,1694775,7.37,124.27,86.41,86.41,63,95,2.15
2,Dingle,PH,1714733,11.00,122.67,82.13,82.13,81,100,3.31
3,Ovsyanka,RU,1495797,55.96,92.55,66.20,66.20,68,33,4.47
4,Polillo,PH,1693125,14.71,121.94,85.05,85.05,77,100,5.10


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [16]:
#Configure URL for API data pull
gmaps.configure(api_key = g_key)

#Store Coordinates for each city
locations = vacay_df[["lat", "long"]]

#Convert and store datapoints of intereest
humidity = vacay_df["humidity"].astype(float)
temp = vacay_df["max_temp"].astype(float)
cloud = vacay_df["cloud_coverage"].astype(float)
wind = vacay_df["wind_speed"].astype(float)

In [17]:
#Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights = humidity,
                                dissipating=False,
                                max_intensity=100,
                                point_radius = 1)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria

In [19]:
#Create new dataframe with key specifications
vacay_df = vacay_df.loc[(vacay_df["max_temp"] <= 70) & 
                        (vacay_df["max_temp"] <= 80) & 
                        (vacay_df["wind_speed"] <= 10) &
                       (vacay_df["cloud_coverage"] == 0)].dropna()


### Hotel Map

In [20]:
#search for hotels vwithin a 5000 meter radius
hotel_df = pd.DataFrame(vacay_df[["city", "country", "lat", "long"]]).reset_index()

# hotel_df["Coordinates"]=""
hotel_df["Hotel Name"] = ""

In [21]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

for index, row in hotel_df.iterrows():
    # retrieve city name and coordinates 
    lat = row["lat"]
    lng = row["long"]
    city_name = row["city"]
    
    # add location param to dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results: {city_name}")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
        print("hotel found: updating dataframe")

        
    except (KeyError, IndexError):
        print("Missing field/result: skipping.")
        
    print(" ")

Retrieving Results: Khatanga
hotel found: updating dataframe
 
Retrieving Results: Saldanha
hotel found: updating dataframe
 
Retrieving Results: Vardø
hotel found: updating dataframe
 
Retrieving Results: Cape Town
hotel found: updating dataframe
 
Retrieving Results: Mossamedes
hotel found: updating dataframe
 
Retrieving Results: Port Alfred
hotel found: updating dataframe
 
Retrieving Results: Qaanaaq
hotel found: updating dataframe
 
Retrieving Results: Port Elizabeth
hotel found: updating dataframe
 
Retrieving Results: Lucapa
Missing field/result: skipping.
 
Retrieving Results: Plettenberg Bay
hotel found: updating dataframe
 
Retrieving Results: Hermanus
hotel found: updating dataframe
 
Retrieving Results: Kaputa
hotel found: updating dataframe
 
Retrieving Results: Spirovo
hotel found: updating dataframe
 
Retrieving Results: Kruisfontein
Missing field/result: skipping.
 
Retrieving Results: Polyarny
hotel found: updating dataframe
 
Retrieving Results: Villa Carlos Paz
hote

In [25]:
hotel_df.dropna()

,index,city,country,lat,long,Hotel Name
0,10,Khatanga,RU,71.97,102.50,"Gostinitsa ""Zapolyar'ye"""
1,12,Saldanha,ZA,-33.01,17.94,Blue Bay Lodge
2,25,Vardø,NO,70.37,31.11,Vardö Apartments
3,26,Cape Town,ZA,-33.93,18.42,Atlanticview Cape Town Boutique Hotel
4,28,Mossamedes,AO,-15.20,12.15,Hotel Chik Chik Namibe
5,57,Port Alfred,ZA,-33.59,26.89,The Halyards Hotel
6,77,Qaanaaq,GL,77.48,-69.36,Qaanaaq Hotel
7,95,Port Elizabeth,ZA,-33.92,25.57,Lemon Tree Lane Guest House
8,104,Lucapa,AO,-8.42,20.74,
9,109,Plettenberg Bay,ZA,-34.05,23.37,Sky Villa Boutique Hotel


In [26]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "long"]]

In [27]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))